In [11]:
# rmsoldie genetic algorithms
import sys
import matplotlib.pyplot as plt
import moviepy.editor as mvp
import numpy as np
import os
import random
import time
import tqdm
from matplotlib.colors import to_rgb
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter
from pathlib import Path
from PIL import Image, ImageDraw, ImageTk
from enum import Enum, EnumMeta, auto
from IPython.display import display, Video, HTML, clear_output

In [12]:
def clamp(lower, higher, val):
    if val < lower:
        val = lower
    elif val > higher:
        val = higher
    return val

In [13]:
class Vesicle():
    def __init__(self, x, y, state_arr):
        global general_gap
        # dir: (wavebool, gapsize)
        directions = ["N", "E", "S", "W"]
        self.empty_table = [(direct, False, general_gap) for direct in directions]
        self.input_table = self.empty_table.copy()
        self.coordinates = (x, y)
        self.state_val = 0
        self.grid = state_arr
            
    def get_input_table(self):
        return self.input_table
    
    def set_grid(self, state_arr):
        self.grid = state_arr
    
    def get_state(self):
        return self.state_val
    
    def set_state(self, new_state):
        self.state_val = new_state
        
    def set_gap(self, gap_direct, gap_size):
        for i in range(len(self.input_table)):
            (direct, wb, _) = self.input_table[i]
            if direct == gap_direct:
                self.input_table[i] = (direct, wb, gap_size)
        
    def update(self, state_arr): #problem
        self.calc_inputs(state_arr)
        self.calc_state()
    
    def calc_inputs(self, state_arr):
        global width, height
        (x, y) = self.coordinates
        gaps = []
        for (_, _, gap) in self.input_table:
            gaps.append(gap)
            
        self.input_table = []         
        if (x-1) in range(height) and (y) in range(width):
            north = state_arr[x-1][y].get_state()
        else:
            north = 0
        
        if (x) in range(height) and (y+1) in range(width):
            east = state_arr[x][y+1].get_state()
        else:
            east = 0
        if (x+1) in range(height) and (y) in range(width):
            south = state_arr[x+1][y].get_state()
        else:
            south = 0
        if (x) in range(height) and (y-1) in range(width):
            west = state_arr[x][y-1].get_state()
        else:
            west = 0
            
        if north in [1, 2, 4, 5, 7, 9, 12, 16]:
            tr = True
        else:
            tr = False
        self.input_table.append(("N", tr, gaps[0]))
            
        if east in [1, 2, 3, 5, 6, 8, 11, 16]:
            tr = True
        else:
            tr = False
        self.input_table.append(("E", tr, gaps[1]))
            
        if south in [2, 3, 4, 8, 9, 10, 14, 16]:
            tr = True
        else:
            tr = False
        self.input_table.append(("S", tr, gaps[2]))
        
        if west in [1, 3, 4, 6, 7, 10, 13, 16]:
            tr = True
        else:
            tr = False
        self.input_table.append(("W", tr, gaps[3]))
        
    def calc_state(self):
        global gap_threshold, lookup_table
        inp = self.input_table
        stimed_directs = []
        
        for (direct, wavebool, gapsize) in inp:
            if wavebool == True and gapsize > gap_threshold:
                stimed_directs.append((direct))
        
        if len(stimed_directs) == 1:
            stimed_directs = (stimed_directs[0],)
        else:
            stimed_directs = tuple(stimed_directs)
        self.state_val = lookup_table[stimed_directs]
        

In [14]:
def build_lookup_tables():
    lookup_table=dict()
    temp = []

    # 0
    temp.append(())
    temp.append(("N",))
    temp.append(("E",))
    temp.append(("S",))
    temp.append(("W",))
    # 5
    temp.append(("N", "E"))
    temp.append(("N", "S"))
    temp.append(("N", "W"))
    temp.append(("E", "S"))
    temp.append(("E", "W"))
    temp.append(("S", "W"))
    # 11
    temp.append(("N", "E", "S"))
    temp.append(("N", "E", "W"))
    temp.append(("N", "S", "W"))
    temp.append(("E", "S", "W"))
    # 15
    temp.append(("N", "E", "S", "W"))
    
    for i in range(len(temp)):
        elem = temp[i]
        lookup_table[elem] = i

    return lookup_table

def stim(x, y):
    global state_arr, width, height
    if x in range(width) and y in range(height):
        state_arr[y][x].set_state(16)
        
def set_gap(x, y, east_wall: bool, gapsize):
    global state_arr, width, height
    if east_wall:
        if x in range(width) and x+1 in range(width) and y in range(height):
            state_arr[y][x].set_gap("E", gapsize)
            state_arr[y][x+1].set_gap("W", gapsize)
        # else - a border
    # south wall
    else:
        if x in range(width) and y in range(height) and y+1 in range(height):
            state_arr[y][x].set_gap("S", gapsize)
            state_arr[y+1][x].set_gap("N", gapsize)
        # else - a border

In [15]:
    
    # output_table=dict()
    # output_table[0] = temp[0]
    # output_table[1] = temp[]
    # output_table[2]
    # output_table[3]
    # output_table[4]
    # output_table[5]
    # output_table[6]
    # output_table[7]
    # output_table[8]
    # output_table[9]
    # output_table[10]
    # output_table[11]
    # output_table[12]
    # output_table[13]
    # output_table[14]
    # output_table[15]

In [16]:
# def get_opposite_dir(dir):
#     opp_dir = (dir+2)%4
#     return opp_dir

# def get_neigh_input(dir, cell_tuple: tuple[Output, list]):
#     dir = get_opposite_dir(dir)
#     _, out_arr = cell_tuple
#     if dir in out_arr:
#         return True
#     else:
#         return False
    
# def calc_new_values(cell_tuple: tuple[Output, list]):
#     # get all 4 neigh values
#     enum, out_arr = cell_tuple
#     if enum in AllOutputs._member_names_:
#         # handle empty cell
#         # handle all outputs
#         pass
    
#     input_dir = enum.value%10
#     # ooh how to handle this...
#     pass

# def update_vesicles(state_arr):
#     num_rows, num_columns = np.shape(state_arr)
#     new_array = []
#     for x in range(num_rows):
#         for y in range(num_columns):
#             # decide new value
                   
            
#             new_array[x, y]
#     return new_array


In [19]:
import copy

def upd_grid(grid):
    old_grid = copy.deepcopy(grid)
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            grid[i][j].update(old_grid)
    return grid

def get_vals_to_print(state_arr):
    num_rows, num_columns = np.shape(state_arr)
    out = ""
    for x in range(num_rows):
        for y in range(num_columns):
            out+=" "+str(state_arr[x][y].get_state()).zfill(2)
        out+="\n"
    return out

In [18]:
frames = 4
sleep_dur = 1.2

general_gap = 0
custom_gap=3
gap_threshold = 0
width, height = 3, 2
lookup_table = build_lookup_tables()
state_arr = np.empty((height, width), dtype=object)
for i in range(height):
    for j in range(width):
        state_arr[i][j] = Vesicle(i, j, state_arr)
# stim
stim(0, 0)
stim(2, 0)

# custom gaps
set_gap(0, 0, True, custom_gap)
set_gap(1, 0, True, custom_gap)
set_gap(1, 0, False, custom_gap)

print(get_vals_to_print(state_arr))
time.sleep(sleep_dur)
for frame in range(frames-1):
    state_arr = upd_grid(state_arr)
    clear_output(wait=True)
    print(get_vals_to_print(state_arr))
    time.sleep(sleep_dur)

 00 00 00
 00 00 00

